In [1]:
import numpy as np
from sympy import *

## Golden Section

#### Question 1 (using number of iterations)

In [2]:
def f(x):
    return x*x

def GS(a,b,n):
    while n!=0:
        t=0.618
        alpha1=a + (1-t)*(b-a)
        alpha2=a + t*(b-a)
    
        if f(alpha1)<f(alpha2):
            b=alpha2
        elif f(alpha1)>f(alpha2):
            a=alpha1
        
        n-=1
        
    return (a+b)/2

In [3]:
a=-5
b=15
n=7

GS(a,b,n)

0.06531008697443558

#### Question 2 (using tolerance)

In [4]:
def f(x):
    return x*(x-1.5)

def GS(a,b,tol):
    t=0.618
    while abs(a-b)>tol:
        alpha1=a + (1-t)*(b-a)
        alpha2=a + t*(b-a)

        if f(alpha1)<f(alpha2):
            b=alpha2
        elif f(alpha1)>f(alpha2):
            a=alpha1

    return (a+b)/2

In [5]:
GS(0,1,0.3)

0.736090516

## Fibonacci Method

In [6]:
def f(x):
    return x*x

def FM(a,b,n):
    l0=b-a
    k=2
    while k!=n:
        alpha1=a + (fibonacci(n-k)/fibonacci(n))*l0
        alpha2=b - (fibonacci(n-k)/fibonacci(n))*l0
        print(alpha1,alpha2)
        
        if f(alpha1)<f(alpha2):
            b=alpha2
        elif f(alpha1)>f(alpha2):
            a=alpha1

        k+=1

    return (a+b)/2

In [7]:
a=-5
b=15
n=6

FM(a,b,n)

5/2 15/2
0 5/2
-5/2 0
0 0


0

## Steepest Descent

In [8]:
def Steepest_Descent(x,f,Q,a,b):
    
    x_old=Matrix([a,b])

    null_mat=Matrix([0,0])

    g_k=Matrix([diff(f,var) for var in x])

    while True:
        g_x=g_k.subs([(x[0],a),(x[1],b)])
    
        if(g_x==null_mat):
            print(f"x:{x_old}")
            break
        d_k=-g_x
    
        alpha_k=((g_x.T * g_x)[0])/((g_x.T * Q * g_x)[0])
    
        x_old+=alpha_k * d_k

        a=x_old[0,0]
        b=x_old[1,0]
        
    return x_old


In [9]:
x=MatrixSymbol('x',2,1)
f=x[0]**2 + x[1]**2 + 2*x[0] + 4*x[1] + 60

Q=Matrix([[2,0],[0,2]])
a=0
b=0

Steepest_Descent(x,f,Q,a,b)

x:Matrix([[-1], [-2]])


Matrix([
[-1],
[-2]])

## Newton Method

In [10]:
def Newton(x,f,a,b):
    
    x_old=Matrix([a,b])
   
    alpha_k=1
    null_mat=Matrix([0,0])

    grad_k=Matrix([diff(f, var) for var in x])
    H_k=hessian(f, tuple(x))
    
    while True:
        g_x=grad_k.subs([(x[0],a),(x[1],b)])
        H_x=H_k.subs([(x[0],a),(x[1],b)])
        H_x_inv=H_x.inv()
            
        if(g_x==null_mat):
            print(f"x:{x_old}")
            break
            
        d_k=H_x_inv * (-g_x)
        
        x_old+=alpha_k * d_k
    
        a=x_old[0,0]
        b=x_old[1,0]
            
    return x_old

In [11]:
x=MatrixSymbol('x',2,1)
f=8*x[0]**2 - 4*x[0]*x[1] + 5*x[1]**2 

a=5
b=2

Newton(x,f,a,b)

x:Matrix([[0], [0]])


Matrix([
[0],
[0]])

## Conjugate Gradient with Direction

In [12]:
def CGwD(x,f,Q,a,b,d):
    null_mat=Matrix([0,0])
    x_old=Matrix([a,b])

    g_k=Matrix([diff(f,var) for var in x])
    
    g_x=g_k.subs([(x[0],a),(x[1],b)])
    if g_x==null_mat:                ## First checking with given x
        return x_old
    
    for i in range(len(d)//2):       ## Since, matrix length means total number of elements
        
        alpha_k=-((g_x.T * d[:,i])[0])/((d[:,i].T * Q * d[:,i])[0])

        x_old+=alpha_k*d[:,i]
        print(x_old)

        a=x_old[0,0]
        b=x_old[1,0]

        g_x=g_k.subs([(x[0],a),(x[1],b)])

        if g_x==null_mat:          ## It starts with 1st iteration
            break

    return x_old

In [13]:
x=MatrixSymbol('x',2,1)
f=x[0]**2 + 2*x[1]**2 + x[0] - x[1] + 1

a=0
b=0

Q=Matrix([[2,0],[0,4]])

d=Matrix([[1,0],[0,1]])

CGwD(x,f,Q,a,b,d)

Matrix([[-1/2], [0]])
Matrix([[-1/2], [1/4]])


Matrix([
[-1/2],
[ 1/4]])

## Conjugate Gradient without Direction

In [14]:
def CGwtD(x,f,Q,a,b):
    x_old=Matrix([a,b])
   
    null_mat=Matrix([0,0])

    grad_k=Matrix([diff(f, var) for var in x])
    
    g_x=grad_k.subs([(x[0],a),(x[1],b)])  # Finding the value of grad
    d_old=-g_x

    if(g_x==null_mat):  # First g(x_0) checking
        return x_old

    while True:
        
        alpha_k= -((g_x.T * d_old)[0]) / ((d_old.T*Q*d_old)[0])
        x_old+=alpha_k * d_old   # updating x
        
        a=x_old[0,0]   # extracting values from new x vector
        b=x_old[1,0]
        
        g_x=grad_k.subs([(x[0],a),(x[1],b)])
        
        if(g_x==null_mat):   # From second g(x) checking
            print(f"x:{x_old}")
            break
        
        beta=((g_x.T*Q*d_old)[0]) / ((d_old.T*Q*d_old)[0])
        d_old=-g_x+beta*d_old
            
    return x_old

In [15]:
x=MatrixSymbol('x',2,1)
f=2*x[0]**2 + x[1]**2 + 2*x[0]*x[1] + x[0] - x[1] 

a=0
b=0

Q=Matrix([[4,2],[2,2]])

CGwtD(x,f,Q,a,b)

x:Matrix([[-1], [3/2]])


Matrix([
[ -1],
[3/2]])

## BFGS Method

In [16]:
from sympy.abc import alpha

def alpha_k(x_k,g_k,H):
    g_x=g_k.subs({x[0]: x_k[0], x[1]: x_k[1]})

    x_next=x_k - alpha * H * g_x
   
    f_alpha=f.subs({x[0]: x_next[0], x[1]: x_next[1]})    # Substitute into f to get f(x_k + alpha * p_k)
    f_alpha=simplify(f_alpha)
    df_dalpha=diff(f_alpha, alpha)    # Differentiate f_alpha w.r.t alpha to find optimal step length
    alpha_star=solve(df_dalpha, alpha)     # Solve df/dalpha = 0 for alpha

    return alpha_star[0]

In [17]:
def BFGS(x,f,Q,a1,a2):
    
    x_old=Matrix([a1,a2])
   
    null_mat=Matrix([0,0])
    B= Matrix.eye(2)   # Initial Hessian approximation (identity)
    H= B.inv()

    grad_k=Matrix([diff(f, var) for var in x])
    y_old=grad_k.subs([(x[0],a1),(x[1],a2)])
    
    ls_x=[]
    ls_y=[]
    i=0
    
    while True:

        print(x_old)
        g_x=y_old

        norm=(g_x[0]**2 + g_x[1]**2)**0.5
    
        if(norm<10e-10):
            print(f"x:{x_old}")
            break
            
        d_k= H*(-g_x)

        alpha_opt=alpha_k(x_old,grad_k,H)
        
        x_new=x_old + alpha_opt*d_k

        ls_x.append(x_new-x_old)
    
        a1=x_new[0,0]
        a2=x_new[1,0]

        y_new=grad_k.subs([(x[0],a1),(x[1],a2)])

        ls_y.append(y_new-y_old)
        
        x_old=x_new
        y_old=y_new

        part1=(ls_y[i]*ls_y[i].T)/((ls_y[i].T*ls_x[i])[0])
        part2=(B*ls_x[i]*ls_x[i].T*B)/((ls_x[i].T*B*ls_x[i])[0])
        B+=part1-part2
        i+=1
        H=B.inv()

    ls=[x_old,i+1]
        
    return ls

In [18]:
x=MatrixSymbol('x',2,1)
f=x[0]**2 + 2.5*x[1]**2 - 3*x[0]*x[1] - x[1] 

a1=0
a2=0

Q=Matrix([[2,-3],[-3,5]])
b=Matrix([[0],[1]])

BFGS(x,f,Q,a1,a2)

Matrix([[0], [0]])
Matrix([[0], [0.200000000000000]])
Matrix([[3.00000000000000], [2.00000000000000]])
x:Matrix([[3.00000000000000], [2.00000000000000]])


[Matrix([
 [3.0],
 [2.0]]),
 3]

In [19]:
## Another Question 2

x=MatrixSymbol('x',2,1)
f=2*x[0]**2 + 4*x[1]**2 - 4*x[0] - 4*x[0]*x[1] + 4 

a1=0
a2=3

Q=Matrix([[4,-4],[-4,8]])
b=Matrix([[4],[0]])

BFGS(x,f,Q,a1,a2)

Matrix([[0], [3]])
Matrix([[26/17], [12/17]])
Matrix([[2], [1]])
x:Matrix([[2], [1]])


[Matrix([
 [2],
 [1]]),
 3]

In [20]:
## Another Question 3

x=MatrixSymbol('x',2,1)
f=2*x[0]**2 + x[1]**2 + 2*x[0]*x[1] + x[0] - x[1] 

a1=0
a2=0

Q=Matrix([[4,2],[2,2]])
b=Matrix([[-1],[1]])

BFGS(x,f,Q,a1,a2)

Matrix([[0], [0]])
Matrix([[-1], [1]])
Matrix([[-1], [3/2]])
x:Matrix([[-1], [3/2]])


[Matrix([
 [ -1],
 [3/2]]),
 3]

### BFGS Method using in-built function

In [21]:
from scipy.optimize import minimize

def f(x):
    return (x[0]**2 + 2.5*x[1]**2 - 3*x[0]*x[1] - x[1])

iterations_details=[]
def callback(x):
    grad=np.array([2*x[0] - 3*x[1],5*x[1] - 3*x[0] - 1])
    iterations_details.append({'x':x.copy(), 'f':f(x), 'grad':grad})

x0=np.array([0,0])
result=minimize(f,x0,method='BFGS',options={'maxiter':100,'disp':True},callback=callback)

print("Optimization Result:")
print(result)
for i,key in enumerate(iterations_details):
    print(f"{i+1}th iteration--> x:{key['x']}, fval:{key['f']}, grad:{key['grad']}")

Optimization terminated successfully.
         Current function value: -1.000000
         Iterations: 5
         Function evaluations: 21
         Gradient evaluations: 7
Optimization Result:
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -0.9999999999999929
        x: [ 3.000e+00  2.000e+00]
      nit: 5
      jac: [-2.384e-07  0.000e+00]
 hess_inv: [[ 5.000e+00  3.000e+00]
            [ 3.000e+00  2.000e+00]]
     nfev: 21
     njev: 7
1th iteration--> x:[-2.98023225e-09  1.99999994e-01], fval:-0.09999999821186059, grad:[-5.99999987e-01 -2.29477880e-08]
2th iteration--> x:[0.33666668 0.402     ], fval:-0.29066556189573595, grad:[-5.32666642e-01 -3.63257209e-08]
3th iteration--> x:[1.05971637 0.83582981], fval:-0.6235299421487055, grad:[-3.88056707e-01 -3.30543255e-08]
4th iteration--> x:[2.79241683 1.87545011], fval:-0.9956909228634037, grad:[-4.15166772e-02  7.28166967e-08]
5th iteration--> x:[3.00000017 2.00000012], fval:-0.9999999999999929